In [2]:
#This is for top user state wise data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/top/user/country/india/state"
Top_user_state_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'State':[], 'Year':[],'Quater':[],'Entity_name':[], 'Registered_users':[]}

for i in Top_user_state_list:
    p_i=path+"/"+i
    Agg_yr=os.listdir(p_i)
    #delhi
    for j in Agg_yr:
        p_j=p_i+"/"+j
        #2018
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+"/"+k 
            #1.json
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['districts']:
              Name=z['name']
              count=z['registeredUsers']
              clm['Entity_name'].append(Name)
              clm['Registered_users'].append(count)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Top_user_state=pd.DataFrame(clm)
print(Top_user_state)

                          State  Year  Quater               Entity_name  \
0     andaman-&-nicobar-islands  2018       1             south andaman   
1     andaman-&-nicobar-islands  2018       1  north and middle andaman   
2     andaman-&-nicobar-islands  2018       1                  nicobars   
3     andaman-&-nicobar-islands  2018       2             south andaman   
4     andaman-&-nicobar-islands  2018       2  north and middle andaman   
...                         ...   ...     ...                       ...   
8291                west-bengal  2024       4                   hooghly   
8292                west-bengal  2024       4                     nadia   
8293                west-bengal  2024       4           purba medinipur   
8294                west-bengal  2024       4         paschim medinipur   
8295                west-bengal  2024       4         paschim bardhaman   

      Registered_users  
0                 5846  
1                  632  
2                  262  

In [3]:
#You need to map your State names to the names used in the GeoJSON file (properties.ST_NM). Otherwise you will get blank figure.
state_name_map = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar Island',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunanchal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli',
    'delhi': 'NCT of Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}
Top_user_state['State'] = Top_user_state['State'].str.strip().str.lower()
Top_user_state['State'] = Top_user_state['State'].replace(state_name_map)

In [ ]:
#creating table to store the above data
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="xxx", #give your user name. Mostly its root
    password="xxx", #give your password
    database="xxx" #give any database name you want
)
cursor=connection.cursor()
query1 = """
        create table Top_User_State(
        State varchar(100),
        Year int,
        Quater int,
        Entity_name varchar(100),
        Registered_Users_Count float)
        """
cursor.execute(query1)

In [12]:
query3 = "drop table Top_User_State"
cursor.execute(query3)

In [14]:
#storing the above in mysql database
#all data must be tuple and must be a list of tuples
data=[]
for index in Top_user_state.index:
    row = Top_user_state.loc[index]
    row = row.values
    row = row[0],float(row[1]),int(row[2]),row[3],int(row[4])
    data.append(row)
query2 = "insert into Top_User_State values (%s,%s,%s,%s,%s)"
cursor.executemany(query2,data)
connection.commit()

In [4]:
#doing data cleaning for Top_user_state
from sklearn.preprocessing import LabelEncoder
#first step check and remove null values
Top_user_state.isna().sum()
Top_user_state.info()
#Second do encoding and dummies to make all values int
enc_col = ['Entity_name']
dummy_col = ['Year','State','Quater']
encoder = LabelEncoder()
Top_user_state['encoded_entity_name'] = encoder.fit_transform(Top_user_state['Entity_name'])
Top_user_state = pd.get_dummies(Top_user_state,columns=dummy_col,dtype=int)
#third removing outliners
Top_user_state['registered_users_count_std']=(Top_user_state['Registered_users']-Top_user_state['Registered_users'].mean())/Top_user_state['Registered_users'].std()
condition1 = Top_user_state['registered_users_count_std']<=3
condition2 = Top_user_state['registered_users_count_std']>=-3
Top_user_state = Top_user_state[(condition1) & (condition2)]
print(Top_user_state)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8296 entries, 0 to 8295
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   State             8296 non-null   object
 1   Year              8296 non-null   object
 2   Quater            8296 non-null   int64 
 3   Entity_name       8296 non-null   object
 4   Registered_users  8296 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 324.2+ KB
                   Entity_name  Registered_users  encoded_entity_name  \
0                south andaman              5846                  289   
1     north and middle andaman               632                  212   
2                     nicobars               262                  209   
3                south andaman              8143                  289   
4     north and middle andaman               911                  212   
...                        ...               ...                  ...   
8291     

In [19]:
#plotting aggregated_transaction_insurance in the form of india map using plotly
import plotly.express as px

fig = px.choropleth(
    Top_user_state,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='State',
    color='Registered_users',
    color_continuous_scale='Reds'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [5]:
#This is for top user 2018 data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/top/user/country/india/2018"
Top_user_2018_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'State':[], 'Year':[],'Quater':[],'State_name':[], 'registered_users':[]}

for i in Top_user_2018_list:
    p_i=path+"/"+i
    Data=open(p_i,'r')
    D=json.load(Data)
    for z in D['data']['districts']:
        Name=z['name']
        count=z['registeredUsers']
        clm['State_name'].append(Name)
        clm['registered_users'].append(count)
        clm['State'].append(i)
        clm['Year'].append(j)
        clm['Quater'].append(int(i.strip('.json')))
#Succesfully created a dataframe
Top_user_2018=pd.DataFrame(clm)
print(Top_user_2018)

     State  Year  Quater          State_name  registered_users
0   1.json  2024       1     bengaluru urban           1922368
1   1.json  2024       1                pune           1211643
2   1.json  2024       1              jaipur            900773
3   1.json  2024       1     mumbai suburban            719300
4   1.json  2024       1           hyderabad            655175
5   1.json  2024       1          rangareddy            635175
6   1.json  2024       1               thane            631864
7   1.json  2024       1           ahmadabad            572811
8   1.json  2024       1  medchal malkajgiri            549429
9   1.json  2024       1       visakhapatnam            497187
10  2.json  2024       2     bengaluru urban           2553312
11  2.json  2024       2                pune           1592293
12  2.json  2024       2              jaipur           1182983
13  2.json  2024       2     mumbai suburban            957633
14  2.json  2024       2               thane           